In [2]:
import os
os.environ['SPARK_HOME'] = "C:/spark"
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.window import *
from pyspark.sql.functions import *

In [4]:
spark = SparkSession.builder.appName("aggregate-app")\
.getOrCreate()
spark

In [22]:
data = [
    (1, '2024-11-01', 100.0), (1, '2024-11-02', 150.0), (1, '2024-11-03', 200.0),
    (1, '2024-11-05', 130.0), (1, '2024-11-07', 80.0), (1, '2024-11-09', 70.0),
    (1, '2024-11-10', 60.0), (1, '2024-11-11', 90.0), (1, '2024-11-13', 110.0),
    (2, '2024-11-01', 50.0),  (2, '2024-11-02', 60.0),  (2, '2024-11-04', 40.0),
    (2, '2024-11-06', 90.0),  (2, '2024-11-08', 100.0), (2, '2024-11-10', 110.0),
    (2, '2024-11-11', 70.0),  (2, '2024-11-13', 130.0), (3, '2024-11-02', 300.0),
    (3, '2024-11-04', 200.0), (3, '2024-11-06', 400.0), (3, '2024-11-08', 150.0),
    (3, '2024-11-10', 250.0), (3, '2024-11-11', 200.0), (3, '2024-11-13', 100.0)
]
columns = ['customer_id', 'transaction_date', 'amount']

transactions = spark.createDataFrame(data, columns)
transactions = transactions.withColumn('transaction_date', col('transaction_date').cast('date'))

transactions.show()

+-----------+----------------+------+
|customer_id|transaction_date|amount|
+-----------+----------------+------+
|          1|      2024-11-01| 100.0|
|          1|      2024-11-02| 150.0|
|          1|      2024-11-03| 200.0|
|          1|      2024-11-05| 130.0|
|          1|      2024-11-07|  80.0|
|          1|      2024-11-09|  70.0|
|          1|      2024-11-10|  60.0|
|          1|      2024-11-11|  90.0|
|          1|      2024-11-13| 110.0|
|          2|      2024-11-01|  50.0|
|          2|      2024-11-02|  60.0|
|          2|      2024-11-04|  40.0|
|          2|      2024-11-06|  90.0|
|          2|      2024-11-08| 100.0|
|          2|      2024-11-10| 110.0|
|          2|      2024-11-11|  70.0|
|          2|      2024-11-13| 130.0|
|          3|      2024-11-02| 300.0|
|          3|      2024-11-04| 200.0|
|          3|      2024-11-06| 400.0|
+-----------+----------------+------+
only showing top 20 rows



In [23]:
date_sum=transactions.groupBy('transaction_date').agg(sum("amount").alias("cummulative_sum"))
date_sum.show()

+----------------+---------------+
|transaction_date|cummulative_sum|
+----------------+---------------+
|      2024-11-02|          510.0|
|      2024-11-01|          150.0|
|      2024-11-05|          130.0|
|      2024-11-03|          200.0|
|      2024-11-07|           80.0|
|      2024-11-09|           70.0|
|      2024-11-11|          360.0|
|      2024-11-10|          420.0|
|      2024-11-13|          340.0|
|      2024-11-04|          240.0|
|      2024-11-06|          490.0|
|      2024-11-08|          250.0|
+----------------+---------------+



In [29]:
window = Window.partitionBy('customer_id').orderBy('transaction_date').rowsBetween(Window.unboundedPreceding, Window.currentRow)
roll_7_avg = Window.partitionBy('customer_id').orderBy('transaction_date').rowsBetween(-6, 0)
cum_transaction = transactions.withColumn('cumulative_amount', sum('amount').over(window))

trans_avg = cum_transaction.withColumn('rolling_avg_amount', avg('amount').over(roll_7_avg))

trans_avg.show()

+-----------+----------------+------+-----------------+------------------+
|customer_id|transaction_date|amount|cumulative_amount|rolling_avg_amount|
+-----------+----------------+------+-----------------+------------------+
|          1|      2024-11-01| 100.0|            100.0|             100.0|
|          1|      2024-11-02| 150.0|            250.0|             125.0|
|          1|      2024-11-03| 200.0|            450.0|             150.0|
|          1|      2024-11-05| 130.0|            580.0|             145.0|
|          1|      2024-11-07|  80.0|            660.0|             132.0|
|          1|      2024-11-09|  70.0|            730.0|121.66666666666667|
|          1|      2024-11-10|  60.0|            790.0|112.85714285714286|
|          1|      2024-11-11|  90.0|            880.0|111.42857142857143|
|          1|      2024-11-13| 110.0|            990.0|105.71428571428571|
|          2|      2024-11-01|  50.0|             50.0|              50.0|
|          2|      2024-1